# Data preparation

## Load

In [ ]:
# Imports necesarios para carga, visualización y particionado de datos
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy.random import RandomState
import joblib
from datetime import datetime

In [ ]:
# Cargar datos desde el archivo CSV
df = pd.read_csv('./Data/penguins.csv')

In [ ]:
# Mostrar el DataFrame (vista rápida)
df

## Clean

In [ ]:
# Inspección inicial y limpieza básica (mostrar estructura y eliminar filas con NA)

print(f"Dimensiones originales: {df.shape}")
print("\nPrimeras filas:")
print(df.head())
print("\nInformación del DataFrame:")
print(df.info())

print("\nValores faltantes por columna:")
print(df.isna().sum())

df_clean = df.dropna()

## Transform

In [ ]:
# Limpieza de columnas irrelevantes y comprobación de NA después de limpiar
df_clean.drop('Unnamed: 0', axis=1, inplace=True)
df_clean = df_clean.drop(columns=["year"])

print(f"\nDimensiones después de dropna y eliminación de 'year': {df_clean.shape}")
print("\nValores faltantes después de limpieza:")
print(df_clean.isna().sum())

## Validate

## Feature Ingineering

In [ ]:
# Preparar variables categóricas mediante one-hot encoding para modelado

categorical_cols = df_clean.select_dtypes(include=["object", "category"]).columns
categorical_cols = [col for col in categorical_cols if col != "species"]

print(f"Columnas categóricas a codificar: {list(categorical_cols)}")

df_encoded = pd.get_dummies(
    df_clean,
    columns=categorical_cols,
    drop_first=False,
    dtype=int,
)

print("\nColumnas después de la codificación:")
print(list(df_encoded.columns))
print(f"\nDimensiones después de codificar: {df_encoded.shape}")

## Split

In [ ]:
# Separar características (X) y objetivo (y) y crear partición train/test estratificada

y = df_encoded["species"]

X = df_encoded.drop(columns=["species"])

print(f"Número de columnas de características: {X.shape[1]}")
print(f"Columnas de características: {list(X.columns)}")
print(f"\nDimensiones de X: {X.shape}  |  Dimensiones de y: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RandomState(42),
    stratify=y,
)

print(f"\nDimensiones del conjunto de entrenamiento: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Dimensiones del conjunto de prueba:       X_test={X_test.shape}, y_test={y_test.shape}")

# Model Creation 

## Build

In [ ]:
# Construir un clasificador RandomForest simple
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

## Train

In [ ]:
# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Validación cruzada en entrenamiento (5-fold) para estimar estabilidad
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1)
print(f"\nCV (5-fold) - accuracy: mean={cv_scores.mean():.4f}, std={cv_scores.std():.4f}")

## Validate

In [ ]:
# Evaluar el modelo en el conjunto de prueba y mostrar gráficas
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:")
print(cm)

# Confusion matrix (heatmap)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Importancias de features (top 10)
try:
    importances = model.feature_importances_
    feat_names = X.columns
    top_idx = np.argsort(importances)[::-1][:10]
    plt.figure(figsize=(8,4))
    sns.barplot(x=importances[top_idx], y=feat_names[top_idx])
    plt.title('Top 10 Feature Importances')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.show()
except Exception as e:
    print("No se pudieron mostrar importancias de features:", e)

# Distribución: reales vs predichos
actual_counts = y_test.value_counts().sort_index()
pred_counts = pd.Series(y_pred).value_counts().reindex(actual_counts.index, fill_value=0)
df_counts = pd.DataFrame({'actual': actual_counts, 'predicted': pred_counts})
ax = df_counts.plot(kind='bar', figsize=(8,4))
ax.set_title('Distribución: reales vs predichos')
ax.set_xlabel('Clase')
ax.set_ylabel('Conteo')
plt.show()

In [ ]:
# Guardar el modelo entrenado en formato pkl
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = f'./Models/model_random_forest_{timestamp}.pkl'
joblib.dump(model, model_path)
print(f"Modelo guardado en: {model_path}")